In [1]:
# this is to allow you to demo how some of the functions work between RDS postgresql and python and df work
import sys
import psycopg2
sys.path.append('../')
from helper import db 

In [2]:
# for selecting data
query = """SELECT * from temp_data limit 10"""
df = db.run_query(query)
df

,id,date,station,longitude,latitude,depth,fish_type,fish_count,region
0,1,2019-01-01,1,None,None,None,None,None,None
1,2,2018-01-01,1,None,None,None,None,None,None


In [4]:
# for inserting a  single row of data
query = """INSERT INTO temp_data (date, station) VALUES ('2019-01-01', 1)
"""
db.run_query(query)

""


In [13]:
# for inserting dataframes!!!! default uses 'append' unless you specify if_exists differently
import pandas as pd
test = {'date': ['2018-01-01'], 'station': [1]}
df = pd.DataFrame.from_dict(test)
df['date'] = pd.to_datetime(df['date'])
db.insert_table(table_name='temp_data', df=df)

,date,station,longitude,latitude,depth,region,fish_type,fish_count
0,2009-08-01,8,-59.652417,47.724500,435.0,NORTH,sand_lances,0.0
1,2009-08-01,9,-59.851500,47.833167,502.0,NORTH,sand_lances,0.0
2,2009-08-01,10,-60.046167,47.752250,513.5,NORTH,sand_lances,0.0
3,2009-08-01,11,-60.635833,47.847167,308.5,NORTH,sand_lances,0.0
4,2009-08-02,12,-60.795750,47.998917,383.5,NORTH,sand_lances,0.0
...,...,...,...,...,...,...,...,...
41140,2018-10-01,105,-61.127333,47.353500,48.0,SOUTH,atlantic_mackerel,3.0
41141,2018-10-01,106,-61.023667,47.513167,41.0,SOUTH,atlantic_mackerel,0.0
41142,2018-10-01,107,-60.665000,47.427167,62.0,SOUTH,atlantic_mackerel,0.0
41143,2018-10-01,108,-60.641833,47.230500,106.0,SOUTH,atlantic_mackerel,0.0


In [23]:
# skeleton query for creating the fish table
fish_query = """
CREATE TABLE fish_data (
id SERIAL NOT NULL,
date date NOT NULL,
station int,
longitude decimal (30,10),
latitude decimal (30,10),
depth real,
fish_type varchar(50),
fish_count real,
region varchar(20),
PRIMARY KEY (id)
)
"""
db.run_query(gtspp_query)

""


In [19]:
# sample query for deleting a row from the table
query = """
delete from fish_data where id=1
"""
# db.run_query(query)

In [50]:
query = """
DROP TABLE fish_data"""
# db.run_query(query)

""


In [19]:
# for inserting dataframes!!!! default uses 'append' unless you specify if_exists differently
import pandas as pd
test = {'date': ['2018-01-01'], 'station': [2], 'id': 1}
test = pd.DataFrame.from_dict(test)
test['date'] = pd.to_datetime(test['date'])
# db.insert_full_replace(table_name='temp_data', df=df)


table_name = 'blah'
for index, val in df.iterrows():
    col_values = [f'`{i}`' for i in df.columns.tolist()]
    val_values = [db._val_format(i) for i in val.values]
    
    replace_statement = [i[0]+'='+i[1] for i in list(zip(col_values, val_values))]

    query = f"INSERT INTO {table_name} ({','.join(col_values)}) VALUES ({','.join(val_values)})) ON DUPLICATE REPLACE ({','.join(replace_statement)})"



2019-01-01
INSERT INTO blah (`id`,`date`,`station`,`longitude`,`latitude`,`depth`,`fish_type`,`fish_count`,`region`) VALUES (1,'2019-01-01 00:00:00.000000',1,NULL,NULL,NULL,NULL,NULL,NULL)) ON DUPLICATE REPLACE (`id`=1,`date`='2019-01-01 00:00:00.000000',`station`=1,`longitude`=NULL,`latitude`=NULL,`depth`=NULL,`fish_type`=NULL,`fish_count`=NULL,`region`=NULL)
2018-01-01
INSERT INTO blah (`id`,`date`,`station`,`longitude`,`latitude`,`depth`,`fish_type`,`fish_count`,`region`) VALUES (2,'2018-01-01 00:00:00.000000',1,NULL,NULL,NULL,NULL,NULL,NULL)) ON DUPLICATE REPLACE (`id`=2,`date`='2018-01-01 00:00:00.000000',`station`=1,`longitude`=NULL,`latitude`=NULL,`depth`=NULL,`fish_type`=NULL,`fish_count`=NULL,`region`=NULL)


In [20]:
df

,id,date,station,longitude,latitude,depth,fish_type,fish_count,region
0,1,2019-01-01,1,None,None,None,None,None,None
1,2,2018-01-01,1,None,None,None,None,None,None


In [25]:
db.run_query('select * from gtspp')

,id,longitude,latitude,position_quality,station_id,measure_time,measure_time_quality,salinity,salinity_quality,depth,depth_quality,temperature,temperature_quality


In [34]:
import pandas as pd

test = {'year': [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], 
        'fish_sum': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100], 'fish_count': [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        'fish_type': ['arctic_cod', 'arctic_cod', 'arctic_cod', 'arctic_cod', 'arctic_cod', 'arctic_cod', 'arctic_cod', 'arctic_cod', 'arctic_cod', 'arctic_cod', 'arctic_cod'],
        'temperature_sum': [150, 250, 350, 450, 550, 650, 750, 850, 950, 1050, 1150], 'temperature_count': [2, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10],
       'salinity_sum': [160, 260, 360, 460, 560, 660, 760, 860, 960, 1060, 1160], 'salinity_count': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]}
test = pd.DataFrame.from_dict(test)

test.to_csv('viz_test.csv', index=False)